In [0]:
create table if not exists bronze_eps01
 (date STRING, stock_symbol STRING, analyst INT, estimated_eps DOUBLE)
 using DELTA
 TBLPROPERTIES(delta.enableChangeDataFeed=true);


In [0]:
 create table if not exists silver_eps01
 (date STRING,stock_symbol STRING, analyst INT,estimated_eps DOUBLE)
 using DELTA
 TBLPROPERTIES(delta.enableChangeDataFeed=true);

In [0]:
 CREATE OR REPLACE TABLE gold_eps01
 (date STRING,stock_symbol STRING, consensus_eps DOUBLE)
 using DELTA
 TBLPROPERTIES(delta.enableChangeDataFeed=true);

In [0]:
%python
#insert the data in python notebook
 
data=[('3/1/2021','a',1,2.2),('3/1/2021','b',2,2.0),('3/1/2021','b',1,1.3),('3/1/2021','c',2,1.2),('3/1/2021','c',1,2.6),('3/1/2021','a',1,2.2)]
columns=['date','stock_symbol','analyst','estimated_eps']
df=spark.createDataFrame(data=data,schema=columns)
 
df.createOrReplaceTempView("bronze_eps01_march_dataset01")

In [0]:
INSERT INTO  bronze_eps01 TABLE bronze_eps01_march_dataset01

In [0]:
SELECT * FROM bronze_eps01

In [0]:
select * from  table_changes("bronze_eps01",1) 

In [0]:
 -- Insert data from table change to silver table

Insert into silver_eps01
select date, stock_symbol, analyst, estimated_eps from  table_changes("bronze_eps01",1)
 

In [0]:
SELECT * from silver_eps01

In [0]:
select * from  table_changes("silver_eps01",1) 

In [0]:
 -- Insert data from table change to gold table

Insert into gold_eps01
select date, stock_symbol, AVG(estimated_eps) as consensus_eps from silver_eps01 group by date, stock_symbol

In [0]:
SELECT * from gold_eps01

In [0]:
select * from  table_changes("gold_eps01",1) 

In [0]:
%python
#Creating another dataset for april for brozetable
data=[('3/1/2021','a',1,4.2),('4/1/2021','b',2,2.0),('4/1/2021','b',1,1.3),('4/1/2021','c',2,1.2),('4/1/2021','c',1,2.6),('4/1/2021','a',1,2.2)]
columns=['date','stock_symbol','analyst','estimated_eps']
df=spark.createDataFrame(data=data,schema=columns)

df.createOrReplaceTempView("bronze_eps01_April_dataset01")

In [0]:
INSERT INTO  bronze_eps01 TABLE bronze_eps01_April_dataset01

In [0]:
select * from  table_changes("bronze_eps01",1) 

In [0]:
MERGE INTO silver_eps01
USING (
  SELECT *
  FROM (
    SELECT *, 
           ROW_NUMBER() OVER (PARTITION BY date, stock_symbol, analyst ORDER BY _commit_version DESC) AS rn
    FROM table_changes('bronze_eps01', 2)
    WHERE _change_type IN ('insert', 'update_postimage')
  ) 
  WHERE rn = 1
) AS bronze_cdf01
ON bronze_cdf01.date = silver_eps01.date
   AND bronze_cdf01.stock_symbol = silver_eps01.stock_symbol
   AND bronze_cdf01.analyst = silver_eps01.analyst
WHEN MATCHED THEN
  UPDATE SET silver_eps01.estimated_eps = bronze_cdf01.estimated_eps
WHEN NOT MATCHED THEN
  INSERT (date, stock_symbol, analyst, estimated_eps)
  VALUES (date, stock_symbol, analyst, estimated_eps);

In [0]:
select * from  table_changes("silver_eps01",2) 

In [0]:
%python
#upload more than file
# Read the CSV file into a DataFrame
df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/Orders.csv")


# Append the same data 5 times to grow the table
delta_table_path = "/FileStore/tables/orders_delta"
for _ in range(100):
    df.write.format("delta").mode("append").save(delta_table_path)


In [0]:
CREATE TABLE ext_orders
USING DELTA
LOCATION '/FileStore/tables/orders_delta';

In [0]:
DESCRIBE DETAIL ext_orders;

In [0]:
OPTIMIZE ext_orders ZORDER BY (CustomerID);

In [0]:
DESCRIBE DETAIL ext_orders;

In [0]:
%python
print("Test")

In [0]:
%python
#upload more than file
# Read the CSV file into a DataFrame
df = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/Orders-2.csv")

In [0]:
%python
display(dbutils.fs.ls("/FileStore/tables/"))

In [0]:
%python
df2 = spark.read.option("header", True).option("inferSchema", True).csv("/FileStore/tables/Orders*")

In [0]:
CREATE OR REPLACE TABLE orders1 (OrderID INT, CustomerID INT, OrderAmount INT) USING DELTA;

In [0]:
-- IllegalAccessException: Cannot access non /Workspace local filesystem path file:/dbfs/FileStore/tables on Shared cluster
COPY INTO orders1
FROM 'file:/dbfs/FileStore/tables/'
FILEFORMAT = CSV
PATTERN = '.*Orders.*\.csv'
FORMAT_OPTIONS (
  'header' = 'true',
  'inferSchema' = 'true'
);

In [0]:
-- DESCRIBE DETAIL ext_orders;
DESCRIBE HISTORY orders1;

In [0]:
CREATE OR REPLACE TABLE orders (   order_id STRING,   customer_id STRING,   order_date DATE,   amount DOUBLE ) USING DELTA;

In [0]:
-- Identify ACID Compliance 
INSERT INTO orders VALUES ('001', 'C01', '2024-06-01', 100.0);

In [0]:
--Compare Data vs Metadata 
DESCRIBE TABLE orders; 


In [0]:
-- Metadata 
SELECT * FROM orders;   -- Data

In [0]:
CREATE OR REPLACE TABLE orders (
  order_id STRING,
  order_date STRING,
  test_date TIMESTAMP GENERATED ALWAYS AS (CAST(order_date AS TIMESTAMP)), -- add one more column to order table after order_date
  amount DOUBLE
)COMMENT 'Main order table';

In [0]:
-- add one more column to order table after order_date, name is date using timestamp format
describe orders


In [0]:
INSERT INTO orders (order_id, order_date, amount) VALUES
('A001', '2024-07-13', 250.0);

In [0]:
select * from orders

In [0]:
CREATE OR REPLACE TABLE orders (
  order_id INT,
  amount DOUBLE,
  order_date DATE
);

In [0]:
INSERT INTO orders VALUES 
(101, 450, '2024-07-01'),
(102, 300, '2024-07-05');

In [0]:
CREATE OR REPLACE TABLE staging_orders (   order_id INT,   amount DOUBLE ); 
INSERT INTO staging_orders VALUES (101, 500), (105, 180);

In [0]:
MERGE INTO orders
USING staging_orders
ON orders.order_id = staging_orders.order_id
WHEN MATCHED THEN
  UPDATE SET orders.amount = staging_orders.amount
WHEN NOT MATCHED THEN
  INSERT (order_id, amount) VALUES (staging_orders.order_id, staging_orders.amount)


In [0]:
SELECT * FROM orders;

In [0]:
%python
# run /Workspace/Repos/Sunil_Prolifics_Repo/DEA-pratice/task01 notebook
# %run /Workspace/Repos/Sunil_Prolifics_Repo/DEA-pratice/task01
result = dbutils.notebook.run("/Workspace/Repos/Sunil_Prolifics_Repo/DEA-pratice/task01", 60, {"name": "Sunil", "score": "95"})
 
print(f"Returned from child: {result}")

In [0]:
%python
result = dbutils.notebook.run("/Workspace/Repos/Sunil_Prolifics_Repo/DEA-pratice/task01", 60, {"name": "Superman", "score": "10000"})
 
print(f"Returned from child: {result}")

In [0]:
%python
%run /Workspace/Repos/Sunil_Prolifics_Repo/DEA-pratice/task01

In [0]:
%python
print(testing)